In [ ]:
pip install cvxopt

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy.spatial.distance import pdist, squareform
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import os
import numpy as np
import pandas as pd 
import os
from math import sqrt,exp
from numpy.linalg import inv,det
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from numpy import asarray
from scipy import stats

In [ ]:
train=pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")
y_train=train['label']
X_train=train.drop(['label'],axis=1)
X_train=np.array(X_train)
y_train=np.array(y_train)
y_train=np.reshape(y_train,(-1,1))

In [ ]:
X=[]
y=[]
for i in range(len(X_train)):
    if y_train[i]==0 or y_train[i]==1:
        X.append(X_train[i])
        y.append(y_train[i])
X=np.array(X)
y=np.array(y)

In [ ]:
y[y==0]=-1

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,stratify=y,test_size=0.3,random_state=42)

In [ ]:
col_mean = x_train.mean(axis=0)
x_train = x_train - col_mean
col_std = x_train.std(axis=0)
for i in range(x_train.shape[1]):
    if(col_std[i] != 0):
        x_train[:,i] = x_train[:,i]/(col_std[i])

In [ ]:
col_mean = x_test.mean(axis=0)
x_test = x_test - col_mean
col_std = x_test.std(axis=0)
for i in range(x_test.shape[1]):
    if(col_std[i] != 0):
        x_test[:,i] = x_test[:,i]/(col_std[i])

In [ ]:
def get_acc(predictions,y_test):
    acc=0
    predictions=np.reshape(predictions,(-1,1))
    y_test=np.reshape(y_test,(-1,1))
    for i in range(len(predictions)):
        if predictions[i]==y_test[i]:
            acc+=1
    return acc/y_test.shape[0]

In [ ]:
def do_pca(X,k):
    # calculate the mean of each column
    M = mean(X.T, axis=1)
    # center columns by subtracting column means
    C = X - M
    # calculate covariance matrix of centered matrix
    V = cov(C.T)
    # eigendecomposition of covariance matrix
    values, vectors = eig(V)
    # K is number of dimensions we want 
    max_vectors = vectors[:k]
    # project data
#     print(max_vectors.shape)
#     P = max_vectors.dot(C.T)
    return max_vectors.T

In [ ]:
cs=[0.001,0.1,1,10,100,1000]
gamma=0.01
threshold=1e-4

In [ ]:
for C in cs:
    m,n = x_train.shape
    y = y_train.reshape(-1,1) * 1.
    x_dash = y * x_train
    H = np.dot(x_dash , x_dash.T) * 1.

    #Converting into cvxopt format
    P = cvxopt_matrix(H)
#     print(P)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.diag(-np.ones(m)), np.identity(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    A = cvxopt_matrix(y.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))
    #Setting solver parameters (change default to decrease tolerance) 
    cvxopt_solvers.options['show_progress'] = False
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10
    #Run solver
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(sol['x'])
    alphas=np.reshape(alphas,(-1,1))
    S=np.where(alphas > threshold)[0]
    temp=alphas[S]*y_train[S]
    w=np.sum(temp*x_train[S],axis=0)
    w=np.reshape(w,(-1,1))
    b=-0.5*(np.max(np.dot(x_train,w)[y_train==-1])+np.min(np.dot(x_train,w)[y_train==1]))
    predict=np.sign(np.dot(x_test,w)+b)
    predict[predict<0]=-1
    print("Value of C : "+str(C))
    print("No. of support vectors : "+str(len(S)))
    print("Accuracy : "+str(get_acc(predict,y_test)))
    print()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
C=0.1
for feats in range(100,800,100):
    vecs=do_pca(x_train,feats)
    new_x_train=np.dot(x_train,vecs)
    print(new_x_train.shape)
    new_x_test=np.dot(x_test,vecs)
    m,n = new_x_train.shape
    y = y_train.reshape(-1,1) * 1.
    x_dash = y * new_x_train
    H = np.dot(x_dash , x_dash.T) * 1.
#     print(H.shape)
    #Converting into cvxopt format
    P = cvxopt_matrix(H)
#     print(P)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((np.diag(-np.ones(m)), np.identity(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m)*C)))
    A = cvxopt_matrix(y.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))
    #Setting solver parameters (change default to decrease tolerance) 
    cvxopt_solvers.options['show_progress'] = False
    cvxopt_solvers.options['abstol'] = 1e-10
    cvxopt_solvers.options['reltol'] = 1e-10
    cvxopt_solvers.options['feastol'] = 1e-10
    #Run solver
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.ravel(sol['x'])
    alphas=np.reshape(alphas,(-1,1))
    S=np.where(alphas > threshold)[0]
    temp=alphas[S]*y_train[S]
    w=np.sum(temp*new_x_train[S],axis=0)
    w=np.reshape(w,(-1,1))
    b=-0.5*(np.max(np.dot(new_x_train,w)[y_train==-1])+np.min(np.dot(new_x_train,w)[y_train==1]))
    predict=np.sign(np.dot(new_x_test,w)+b)
    predict[predict<0]=-1
    print("No. of features : "+str(feats))
    print("No. of support vectors : "+str(len(S)))
    print("Accuracy : "+str(get_acc(predict,y_test)))
    print()